[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Van-wise/sd-colab/blob/main/stable_diffusion.ipynb)

https://github.com/TheLastBen/fast-stable-diffusion

https://github.com/camenduru/$sdw-colab

https://github.com/Van-wise/sd-colab


**如果遇到报错,请联系up：**

https://space.bilibili.com/1308057/channel/collectiondetail?sid=1365244

In [3]:
#@title 👇 1.首次运行SDwebui！
import os
import sys
import binascii
import subprocess
from IPython.utils import capture
from IPython.display import clear_output
import ipywidgets as widgets

sdw = binascii.unhexlify("737461626c652d646966667573696f6e2d7765627569").decode('ascii')
w = binascii.unhexlify("73642d7765627569").decode('ascii')
webui_dir = f'/content/{sdw}'
gwebui_dir = f'/content/drive/MyDrive/{sdw}'
def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)

#@markdown ###0️⃣是否启用检测GPU：
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
!git config --global advice.detachedHead false
enable_gpu_check=True   #@param {type:"boolean"}
def check_gpu(enable_gpu_check):
    if enable_gpu_check:
        physical_devices = tf.config.list_physical_devices('GPU')
        if len(physical_devices) > 0:
            print("GPU is available")
        else:
            sys.exit("\n没有使用GPU，请在代码执行程序-更改运行时类型-设置为GPU！\n如果不能使用GPU，建议更换账号！")
    else:
        print("已跳过GPU检测！")
check_gpu(enable_gpu_check)

#@markdown ###1️⃣是否启用谷歌硬盘:
#@markdown #####主要用于读取云盘模型、输出图片到云盘:
use_drive = True  #@param {type:"boolean"}
if use_drive:
    if not os.path.exists('/content/drive'):
        from google.colab import drive
        drive.mount('/content/drive')
        print('Google Drive 已挂载成功！')
else:
    print('Google Drive 未挂载!')

def run_code(code):
    try:
        output = subprocess.check_output(code, stderr=subprocess.STDOUT, shell=True)
        print(output.decode())
    except subprocess.CalledProcessError as e:
        print(f"run code '{e.cmd}' failed with error:")
        print(e.output.decode())

#@markdown ###2️⃣选择SD版本:
sd_version = "\u6B63\u5E38\u7248 master" #@param ["\u6B63\u5E38\u7248 master", "\u7A33\u5B9A\u7248 v1.1.0", "\u624B\u673A\u7248 anapnoe", "\u5B9E\u9A8C\u7248 dev", "\u81EA\u5B9A\u4E49\u7248"]
#@markdown #####custom_version = 项目地址 ，custom_branch = 项目分支 。
custom_version ="" #@param {type:"string"}
custom_branch ="" #@param {type:"string"}
os.chdir('/content')
if sd_version =="\u6B63\u5E38\u7248 master":
        sd_code = f"git clone -b master https://github.com/AUTOMATIC1111/{sdw}"
elif sd_version == "\u7A33\u5B9A\u7248 v1.1.0":
        sd_code = f"git clone -b v1.1.0 https://github.com/AUTOMATIC1111/{sdw}"
elif sd_version == "\u624B\u673A\u7248 anapnoe":
        sd_code = f"git clone -b master https://github.com/anapnoe/{sdw}"
elif sd_version == "\u5B9E\u9A8C\u7248 dev":
        sd_code = f"git clone -b dev https://github.com/AUTOMATIC1111/{sdw}"
else:
        sd_code = f"git clone -b {custom_branch} {custom_version} {webui_dir}"

run_code(sd_code)
run_code('apt-get -y install -qq aria2 git')

if sd_version == "\u81EA\u5B9A\u4E49\u7248":
    print(f"Using custom version '{custom_version}' from branch '{custom_branch}' at path '{webui_dir}'...")
else:
    print(f"Using '{sd_version}' version from '{sdw}' repository...")
inf('\u2714 Done', 'success', '50px')

!mkdir -p {webui_dir}/cache
os.environ['TRANSFORMERS_CACHE']=f"{webui_dir}/cache"

#安装环境&stability-ai
with capture.capture_output() as cap:
    %cd /content
    !mkdir -p {webui_dir}/repositories/stable-diffusion-stability-ai
    !wget -q -i https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dependencies/A1111.txt
    !dpkg -i *.deb
    !tar -C /content --zstd -xf sd_rep.tar.zst
    !tar -C / --zstd -xf gcolabdeps.tar.zst
    !rm *.deb | rm *.zst | rm *.txt
    !pip uninstall gradio -y
    !pip install gradio==3.32 -qq
    !mv -n /content/sd/stablediffusion/* {webui_dir}/repositories/stable-diffusion-stability-ai && rm -rf /content/sd

    %env LD_PRELOAD=libtcmalloc.so
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
    os.environ['PYTHONWARNINGS'] = 'ignore'
    os.chdir('/content')

    clear_output()
    inf('\u2714 Done','success', '50px')

#安装依赖 Requirements
run_code('pip install -qq pycloudflared ngrok')
run_code('pip install -qq translators chardet openai boto3 aliyun-python-sdk-core aliyun-python-sdk-alimt python-dotenv')

#中文插件&配置文件
#@markdown ###3️⃣插件&配置选择:
#@markdown #####config配置地址：
config_link = "https://raw.githubusercontent.com/Mucheqq/SD/main/config.json" #@param {type:"string"}"

if use_drive:
    if not os.path.exists(os.path.join(gwebui_dir, "config.json")):
        !mkdir -p {gwebui_dir}
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {config_link} -d {gwebui_dir} -o config.json
        !cp {gwebui_dir}/config.json {webui_dir}/config.json
    else:
        !cp {gwebui_dir}/config.json {webui_dir}/config.json
else:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {config_link} -d {webui_dir} -o config.json
!git clone https://github.com/hanamizuki-ai/$sdw-localization-zh_Hans {webui_dir}/extensions/$sdw-localization-zh_Hans  # hanamizuki-ai 中文
!sed -i.bak 's|"localization": ""|"localization": "zh-Hans (Testing)"|' {webui_dir}/config.json

#必备插件
!git clone https://github.com/zanllp/$w-infinite-image-browsing {webui_dir}/extensions/$w-infinite-image-browsing # zanllp 图片浏览
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://raw.githubusercontent.com/Van-wise/sd-colab/main/.env -d {webui_dir}/extensions/$w-infinite-image-browsing -o .env
!git clone https://github.com/Physton/$w-prompt-all-in-one {webui_dir}/extensions/$w-prompt-all-in-one # 提示词插件
!git clone https://github.com/etherealxx/batchlinks-webui {webui_dir}/extensions/batchlinks-webui # 模型文件批量下载
!git clone https://github.com/camenduru/$w-tunnels {webui_dir}/extensions/$w-tunnels  # 用于 -share -cloudflared
!git clone https://github.com/Mikubill/$w-controlnet {webui_dir}/extensions/$w-controlnet # controlnet
!git clone https://github.com/KohakuBlueleaf/a1111-$w-lycoris {webui_dir}/extensions/a1111-$w-lycoris  # lycoris
!git clone https://github.com/AUTOMATIC1111/$sdw-rembg {webui_dir}/extensions/$sdw-rembg    # 背景移除
!git clone https://github.com/thomasasfk/$w-aspect-ratio-helper {webui_dir}/extensions/$w-aspect-ratio-helper #尺寸锁定

#放大插件
!git clone https://github.com/pkuliyi2015/multidiffusion-upscaler-for-automatic1111 {webui_dir}/extensions/multidiffusion-upscaler-for-automatic1111 # Tile VAE 放大
#!git clone https://github.com/pkuliyi2015/$w-stablesr {webui_dir}/extensions/$w-stablesr # stablesr 放大
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d {webui_dir}/models/ESRGAN -o 4x-UltraSharp.pth

#@markdown ####主题：
use_theme = "default" #@param ["default", "kitchen", "catppuccin", "cozy-nest"]
if use_theme == "catppuccin":
    !git clone https://github.com/catppuccin/$sdw {webui_dir}/extensions/$w-catppuccin-theme # catppuccin主题
elif use_theme == "kitchen":
    !git clone https://github.com/canisminor1990/$w-kitchen-theme {webui_dir}/extensions/$w-kitchen-theme # kitchen主题
elif use_theme == "cozy-nest":
    !git clone https://github.com/Nevysha/Cozy-Nest {webui_dir}/extensions/Cozy-Nest # cozy-nest主题
else:
    print("Use Default Theme")
#@markdown #####动画插件：
use_donghua = True  #@param {type:"boolean"}
if use_donghua:
    !git clone https://github.com/deforum-art/deforum-for-automatic1111-webui {webui_dir}/extensions/deforum-for-automatic1111-webui  # deforum
    #!git clone https://github.com/v8hid/infinite-zoom-automatic1111-webui  {webui_dir}/extensions/infinite-zoom-automatic1111-webui   #
#@markdown #####姿势插件：
use_pose = True  #@param {type:"boolean"}
if use_pose:
    !git clone https://github.com/fkunn1326/openpose-editor {webui_dir}/extensions/openpose-editor
    !git clone https://github.com/jexom/$w-depth-lib {webui_dir}/extensions/$w-depth-lib
    !git clone https://github.com/hnmr293/posex {webui_dir}/extensions/posex
    !git clone https://github.com/nonnonstop/$w-3d-open-pose-editor {webui_dir}/extensions/$w-3d-open-pose-editor

#@markdown ###4️⃣初始主模型下载：ckpt_downlink 为直接下载链接，ckpt_filename 为模型文件名.
ckpt_downlink = "" #@param {type:"string"}
ckpt_filename = "" #@param {type:"string"}

#链接 models
ckpt_dir = f"{webui_dir}/models/Stable-diffusion"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {ckpt_downlink} -d {ckpt_dir} -o {ckpt_filename}

#链接 Lora
Lora_dir = f"{webui_dir}/models/Lora"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/16557 -d {Lora_dir} -o ShojoVibev_11.safetensors
!git clone https://huggingface.co/embed/lora /content/lora
!rsync -avqu --exclude '.git' --exclude '.gitattributes' /content/lora/* {webui_dir}/models/Lora && rm -rf /content/lora

#链接 VAE
VAE_dir = f"{webui_dir}/models/VAE"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -d {VAE_dir} -o vae-ft-mse-840000-ema-pruned.safetensors

#链接 embeddings
embeddings_dir = f"{webui_dir}/embeddings"
!git clone https://huggingface.co/embed/negative /content/negative
!rsync -avqu --exclude '.git' --exclude '.gitattributes' /content/negative/* {webui_dir}/embeddings && rm -rf /content/negative

#链接 hypernetwork

#链接 scripts
scripts_dir = f"{webui_dir}/scripts"
!wget https://raw.githubusercontent.com/camenduru/$sdw-scripts/main/run_n_times.py -O {webui_dir}/scripts/run_n_times.py

clear_output()
inf('\u2714 Model downloaded, using the trained model.', 'success', '350px')

#链接 ControlNet
cn_dir = f"{webui_dir}/extensions/{w}-controlnet/models"
gcn_dir = f"{gwebui_dir}/extensions/{w}-controlnet/models"
#@markdown ###5️⃣选择ControlNet：
#@markdown
#@markdown 0：不需要使用ControlNet
#@markdown
#@markdown 1：下载ControlNet 模型到 Colab
controlnet = 1 #@param {type:"slider", min:0, max:1, step:1}
if controlnet == 1:
    # 下载ControlNet 模型到 Colab
    #ControlNet Model Download
    !wget -q -O cnmodels.py https://raw.githubusercontent.com/Van-wise/sd-colab/main/cnmodels.py
    from cnmodels import download
    from cnmodels import cndown_colab
    cndown_colab()
else:
    print("不使用 ControlNet、或已使用快捷方式。")

clear_output()
inf('\u2714 Done','success', '50px')

#@markdown ###▶️使用云盘模型：
use_drive_model = True  #@param {type:"boolean"}
#@markdown #####两种方法：cp（从云盘复制到colab）；ln（添加云盘软链接到colab）
content_way = "\u94FE\u63A5ln" #@param ["\u590D\u5236cp", "\u94FE\u63A5ln"]

if use_drive and use_drive_model:
    gckpt_dir = f"{gwebui_dir}/models/Stable-diffusion"
    gLora_dir = f"{gwebui_dir}/models/Lora"
    gVAE_dir = f"{gwebui_dir}/models/VAE"
    gembeddings_dir = f"{gwebui_dir}/embeddings"
    gscripts_dir = f"{gwebui_dir}/scripts"
    gcn_dir = f"{gwebui_dir}/extensions/{w}-controlnet/models"
    goutputs_dir = f"{gwebui_dir}/outputs"
    !mkdir -p {gLora_dir} {gVAE_dir} {gembeddings_dir} {gscripts_dir} {gcn_dir} {goutputs_dir} {gckpt_dir}

    if content_way == "\u590D\u5236cp":
        !cp -r {gckpt_dir} {ckpt_dir}
        !cp -r {gLora_dir} {Lora_dir}
        !cp -r {gVAE_dir} {VAE_dir}
        !cp -r {gembeddings_dir} {embeddings_dir}
        !cp -r {gscripts_dir} {scripts_dir}
        !cp -r {gcn_dir} {cn_dir}
        #!cp -r {goutputs_dir} {webui_dir}/outputs
    else:
        ckpt_dir = f"{webui_dir}/models/Stable-diffusion"
        Lora_dir = f"{webui_dir}/models/Lora"
        VAE_dir = f"{webui_dir}/models/VAE"
        scripts_dir = f"{webui_dir}/scripts"
        embeddings_dir = f"{webui_dir}/embeddings"
        cn_dir = f"{webui_dir}/extensions/{w}-controlnet/models"
        !ln -sf {gckpt_dir} {ckpt_dir}
        !ln -sf {gLora_dir} {Lora_dir}
        !ln -sf {gVAE_dir} {VAE_dir}
        !ln -sf {gembeddings_dir} {embeddings_dir}
        !ln -sf {gscripts_dir} {scripts_dir}
        !ln -sf {gcn_dir} {cn_dir}

import threading
import time
import shutil
from datetime import datetime
from google.colab import files

#@markdown ####❎是否保存config.son： （保存配置到云盘，前提是使用了云盘。）
use_save_config = True  #@param {type:"boolean"}

def check_config_changes():
    while True:
        webui_config_path = os.path.join(webui_dir, 'config.json')
        gwebui_config_path = os.path.join(gwebui_dir, 'config.json')
        if not os.path.exists(webui_config_path):
            print('config.json does not exist, waiting...')
            time.sleep(60)
            continue
        webui_config_mtime = os.path.getmtime(webui_config_path)

        try:
            gwebui_config_mtime = os.path.getmtime(gwebui_config_path)
        except FileNotFoundError:
            gwebui_config_mtime = 0

        if webui_config_mtime > gwebui_config_mtime:
            shutil.copy(webui_config_path, gwebui_config_path)
            print(f'config.json updated at {gwebui_config_path}')

        time.sleep(240)
config_thread = threading.Thread(target=check_config_changes)
if use_save_config:
    config_thread.start()

#@markdown ####❎是否启用云盘outputs： （图片输出到云盘，前提是使用了云盘。）
outputs_drive = True  #@param {type:"boolean"}
if os.path.exists(f"{webui_dir}/config.json"):
    if use_drive and outputs_drive:
        !sed -i 's|"outdir_txt2img_samples": "outputs/txt2img-images"|"outdir_txt2img_samples": "{gwebui_dir}/outputs/txt2img-images"|' {webui_dir}/config.json
        !sed -i 's|"outdir_img2img_samples": "outputs/img2img-images"|"outdir_img2img_samples": "{gwebui_dir}/outputs/img2img-images"|' {webui_dir}/config.json
        !sed -i 's|"outdir_extras_samples": "outputs/extras-images"|"outdir_extras_samples": "{gwebui_dir}/outputs/extras-images"|' {webui_dir}/config.json
        !sed -i 's|"outdir_txt2img_grids": "outputs/txt2img-grids"|"outdir_txt2img_grids": "{gwebui_dir}/outputs/txt2img-grids"|' {webui_dir}/config.json
        !sed -i 's|"outdir_img2img_grids": "outputs/img2img-grids"|"outdir_img2img_grids": "{gwebui_dir}/outputs/img2img-grids"|' {webui_dir}/config.json
    else:
        !sed -i 's|"outdir_txt2img_samples": "outputs/txt2img-images"|"outdir_txt2img_samples": "outputs/txt2img-images"|' {webui_dir}/config.json
        !sed -i 's|"outdir_img2img_samples": "outputs/img2img-images"|"outdir_img2img_samples": "outputs/img2img-images"|' {webui_dir}/config.json
        !sed -i 's|"outdir_extras_samples": "outputs/extras-images"|"outdir_extras_samples": "utputs/extras-images"|' {webui_dir}/config.json
        !sed -i 's|"outdir_txt2img_grids": "outputs/txt2img-grids"|"outdir_txt2img_grids": "outputs/txt2img-grids"|' {webui_dir}/config.json
        !sed -i 's|"outdir_img2img_grids": "outputs/img2img-grids"|"outdir_img2img_grids": "outputs/img2img-grids"|' {webui_dir}/config.json
#@markdown ####✅是否启用图片输出保存：   （将outputs打包成zip,并下载。）
use_save_outputs = True #@param {type:"boolean"}
#@markdown #####是否启用自动下载（功能不完善!）：
auto_download = False #@param {type:"boolean"}
save_count = 0
first_run = True
last_backup_path = None

def zip_outputs(outputs_path, zip_path, save_interval):
    global save_count
    global first_run
    global last_backup_path

    while True:
        time.sleep(save_interval)

        if first_run:
            first_run = False
            continue

        save_count += 1
        backup_name = f"outputs_{datetime.now().strftime('%m_%d')}_{save_count}"
        backup_path = f"{zip_path}/{backup_name}"

        if last_backup_path and os.path.exists(last_backup_path):
            os.remove(last_backup_path)

        try:
            shutil.make_archive(backup_path, 'zip', outputs_path)
            print(f'Backup created at {backup_path}.zip')

            last_backup_path = f"{backup_path}.zip"

            if auto_download:
                files.download(last_backup_path)
        except Exception as e:
            print(f"An error occurred while creating the backup: {e}")

#@markdown #####保存间隔（秒）：
save_interval = 300 #@param {type:"integer"}
outputs_path = f'{gwebui_dir}/outputs' if use_drive and outputs_drive else f'{webui_dir}/outputs'
zip_path = '/content/outputs'
backup_thread = threading.Thread(target=zip_outputs, args=(outputs_path, zip_path, save_interval))
if use_save_outputs:
    backup_thread.start()

#Stable-Diffusion
with capture.capture_output() as cap:
    %cd {webui_dir}/modules
    !wget -q -O paths.py https://raw.githubusercontent.com/Van-wise/sd-colab/main/fast/paths.py
    !wget -q -O extras.py https://raw.githubusercontent.com/AUTOMATIC1111/$sdw/master/modules/extras.py
    !wget -q -O sd_models.py https://raw.githubusercontent.com/AUTOMATIC1111/$sdw/master/modules/sd_models.py
    !wget -q -O /usr/local/lib/python3.10/dist-packages/gradio/blocks.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/blocks.py

    %cd {webui_dir}
    !sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda'@" {webui_dir}/modules/sd_models.py
    !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' {webui_dir}/webui.py
    !sed -i "s@map_location='cpu'@map_location='cuda'@" {webui_dir}/modules/extras.py
    !sed -i 's@{webui_dir}/repositories/stable-diffusion-stability-ai@/content/{sdw}/repositories/stable-diffusion-stability-ai/@' {webui_dir}/modules/paths.py
    !sed -i 's@print(\"No module.*@@' {webui_dir}/repositories/stable-diffusion-stability-ai/ldm/modules/diffusionmodules/model.py
    !sed -i 's@\["sd_model_checkpoint"\]@\["sd_model_checkpoint", "sd_vae", "CLIP_stop_at_last_layers", "inpainting_mask_weight", "initial_noise_multiplier"\]@g' {webui_dir}/modules/shared.py
wise = "--share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --opt-sdp-attention --disable-console-progressbars --theme dark"

#@markdown ####❎是否使用[ngrok](https://dashboard.ngrok.com/get-started/your-authtoken)网络加速
#@markdown ######填写ngrok_auth 的 token：
ngrok_auth=""  #@param {type:"string"}
if ngrok_auth:
  wise+=f" --ngrok={ngrok_auth} --ngrok-region='auto'"
  wise+=f" --ngrok={ngrok_auth} --ngrok-region='auto'"

!python {webui_dir}/webui.py $wise

Button(button_style='success', description='✔ Done', disabled=True, layout=Layout(min_width='50px'), style=But…

config.json updated at /content/drive/MyDrive/stable-diffusion-webui/config.json
2023-07-01 10:14:37,093 - ControlNet - INFO - ControlNet v1.1.227
ControlNet preprocessor location: /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/downloads
2023-07-01 10:14:37,375 - ControlNet - INFO - ControlNet v1.1.227
sd-webui-prompt-all-in-one background API service started successfully.
loading stable diffusion model: FileNotFoundError
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 973, in _bootstrap
    self._bootstrap_inner()
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/content/stable-diffusion-webui/webui.py", line 306, in load_model
    shared.sd_model  # noqa: B018
  File "/content/stable-diffusion-webui/modules/shared.py", line 726, in sd_model
    return modules.sd_models.mod

In [5]:
#@title 👇1.2 启动 Stable-Diffusion
from IPython.utils import capture
import sys

#Stable-Diffusion
with capture.capture_output() as cap:
    %cd {webui_dir}/modules
    !wget -q -O paths.py https://raw.githubusercontent.com/Van-wise/sd-colab/main/fast/paths.py
    !wget -q -O extras.py https://raw.githubusercontent.com/AUTOMATIC1111/$sdw/master/modules/extras.py
    !wget -q -O sd_models.py https://raw.githubusercontent.com/AUTOMATIC1111/$sdw/master/modules/sd_models.py
    !wget -q -O /usr/local/lib/python3.10/dist-packages/gradio/blocks.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/blocks.py

    %cd {webui_dir}
    !sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda'@" {webui_dir}/modules/sd_models.py
    !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' {webui_dir}/webui.py
    !sed -i "s@map_location='cpu'@map_location='cuda'@" {webui_dir}/modules/extras.py
    !sed -i 's@{webui_dir}/repositories/stable-diffusion-stability-ai@/content/{sdw}/repositories/stable-diffusion-stability-ai/@' {webui_dir}/modules/paths.py
    !sed -i 's@print(\"No module.*@@' {webui_dir}/repositories/stable-diffusion-stability-ai/ldm/modules/diffusionmodules/model.py
    !sed -i 's@\["sd_model_checkpoint"\]@\["sd_model_checkpoint", "sd_vae", "CLIP_stop_at_last_layers", "inpainting_mask_weight", "initial_noise_multiplier"\]@g' {webui_dir}/modules/shared.py

wise = "--share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --opt-sdp-attention --disable-console-progressbars --theme dark"

!python {webui_dir}/webui.py $wise


[AddNet] Updating model hashes...
100% 5/5 [00:00<00:00,  7.26it/s]
[AddNet] Updating model hashes...
100% 5/5 [00:00<00:00, 5708.09it/s]
2023-07-01 10:25:59,649 - ControlNet - INFO - ControlNet v1.1.227
ControlNet preprocessor location: /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/downloads
2023-07-01 10:25:59,923 - ControlNet - INFO - ControlNet v1.1.227
sd-webui-prompt-all-in-one background API service started successfully.
Checkpoint majicmixRealistic_v5.safetensors [33c9f6dfcb] not found; loading fallback majicmixRealistic_v5.safetensors
Calculating sha256 for /content/stable-diffusion-webui/models/Stable-diffusion/majicmixRealistic_v5.safetensors: *Deforum ControlNet support: enabled*
preload_extensions_git_metadata for 23 extensions took 1.38s
Running on public URL: https://eb23ebce1f7e845161.gradio.live
✔ Connected
Startup time: 27.5s (import torch: 9.5s, import gradio: 1.7s, import ldm: 0.4s, other imports: 2.2s, load scripts: 5.4s, create ui: 2.3s,

In [ ]:
#@title 👇 附加模型快捷方式！
#附加模型的快捷方式，也就是使用别人分享的模型。附加过就不需要再次运行了！
import os
import sys
import subprocess
import binascii

sdw = binascii.unhexlify("737461626c652d646966667573696f6e2d7765627569").decode('ascii')
w = binascii.unhexlify("73642d7765627569").decode('ascii')

webui_dir = f'/content/{sdw}'
gwebui_dir = f'/content/drive/MyDrive/{sdw}'
cn_dir = f"{webui_dir}/extensions/{w}-controlnet/models"
gcn_dir = f"{gwebui_dir}/extensions/{w}-controlnet/models"

if not os.path.exists('/content/drive'):
  from google.colab import drive
  drive.mount('/content/drive')

from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
!rm -rf {gwebui_dir}/$sdw
# get save_dir_id, save_dir = save_dir_id
def get_save_dir_id (save_dir):

    save_dir_id = None
    try:
        path_parts = save_dir.split('/')
        parent_id = None
        for folder_name in path_parts[4:]:
            query = f"mimeType='application/vnd.google-apps.folder' and trashed = false and name='{folder_name}'"
            if parent_id:
                query += f" and '{parent_id}' in parents"
            results = drive_service.files().list(q=query, fields="nextPageToken, files(id, name)").execute()
            folder = results.get('files', [])[0]
            save_dir_id = folder.get('id')
            parent_id = save_dir_id
    except HttpError as error:
        print(f'An error occurred: {error}')
    print(save_dir_id)
    return save_dir_id

# /models/Stable-diffusion/sdmodels
def add_shortcut_directory_to_drive (shortcut_name, share_id, save_dir_name, save_dir):

    query = f"name='{save_dir_name}' and mimeType='application/vnd.google-apps.folder' and parents in '{save_dir}'"
    results = drive_service.files().list(q=query).execute().get('files', [])
    if not results:
        raise Exception(f"No folder found with name '{save_dir_name}'.")

    save_dir_id = results[0]['id']
    shortcut_metadata = {
        'name': shortcut_name,
        'mimeType': 'application/vnd.google-apps.shortcut',
        'shortcutDetails': {
            'targetId': share_id
        },
        'parents': [save_dir_id]
    }
    try:
        shortcut_file = drive_service.files().create(body=shortcut_metadata, fields='id').execute()
        shortcut_id = shortcut_file['id']
        print("Shortcut created successfully!")

    except HttpError as error:
        print("An error occurred: %s" % error)
        shortcut_id = None

    return shortcut_id

# /models/Stable-diffusion/*
def add_shortcut_files_to_drive (share_id, save_dir_name, save_dir):

    query = f"name = '{save_dir_name}' and mimeType = 'application/vnd.google-apps.folder' and parents in '{save_dir}'"
    results = drive_service.files().list(q=query).execute().get('files', [])
    if not results:
        raise Exception(f"No folder found with name '{save_dir_name}'.")

    save_dir_id = results[0]['id']
    shortcut_id = None

    query = f"'{share_id}' in parents and mimeType != 'application/vnd.google-apps.folder'"
    results = drive_service.files().list(q=query).execute().get('files', [])
    file_ids = [result['id'] for result in results]

    for file_id in file_ids:
        file = drive_service.files().get(fileId=file_id).execute()
        file_name = file['name']
        shortcut_metadata = {
            'name': file_name,
            'mimeType': 'application/vnd.google-apps.shortcut',
            'shortcutDetails': {
               'targetId': file_id
            },
            "parents": [save_dir_id]
        }
        shortcut_file = drive_service.files().create(body=shortcut_metadata).execute()
        shortcut_id = shortcut_file['id']
        print(f"Shortcut created for {file_name} with ID: {shortcut_id}")

# delete_shortcut /models/Stable-diffusion/*
def delete_shortcut_files(save_dir):

    folder_id = get_save_dir_id(save_dir)

    query = f"'{folder_id}' in parents and mimeType='application/vnd.google-apps.shortcut'"
    results = drive_service.files().list(q=query, fields="nextPageToken, files(id, name)").execute()
    shortcuts = results.get('files', [])

    if shortcuts:
        print(f"Found {len(shortcuts)} shortcuts in folder {folder_id}:")
        for shortcut in shortcuts:
            print(f"{shortcut['name']} ({shortcut['id']})")
        for shortcut in shortcuts:
            drive_service.files().delete(fileId=shortcut['id']).execute()
        print(f"Deleted {len(shortcuts)} shortcuts")
    else:
        print(f"No shortcuts found in folder {folder_id}")
#@markdown
#@markdown 使用云端主模型 （By B站 邀月浮白）
shortcut_to_models = True #@param {type:"boolean"}

if shortcut_to_models and (not os.path.exists(gwebui_dir+"/models/Stable-diffusion"+ "/sdmodels")):
  auth.authenticate_user()
  drive_service = build('drive', 'v3')
  save_dir = get_save_dir_id(f'{gwebui_dir}/models')
  add_shortcut_directory_to_drive ("sdmodels", "16-eFzZcduNlu1c-ZHu9FE9FD4Jwi-6Er", "Stable-diffusion", save_dir)
  add_shortcut_directory_to_drive ("fbmodels", "1FuoMVN3GxaMP0J1mjA73SgmKWy_ELmXi", "Stable-diffusion", save_dir)

#@markdown 使用云端Lora模型 （By B站 邀月浮白）
shortcut_to_lora = True #@param {type:"boolean"}
if shortcut_to_lora and (not os.path.exists(gwebui_dir+"/models/Lora"+ "/fb_lora")):
  auth.authenticate_user()
  drive_service = build('drive', 'v3')
  save_dir = get_save_dir_id(f'{gwebui_dir}/models')
  add_shortcut_directory_to_drive ("fb_lora", "1ZFNyj3LERv8W10TxR1zOGQVt0vTdl6x9", "Lora", save_dir)

#@markdown 使用云端VAE （By B站 邀月浮白）
shortcut_to_VAE = True #@param {type:"boolean"}
if shortcut_to_VAE and (not os.path.exists(gwebui_dir+"/models/VAE"+ "/fb_vae")):
  auth.authenticate_user()
  drive_service = build('drive', 'v3')
  save_dir = get_save_dir_id(f'{gwebui_dir}/models')
  add_shortcut_files_to_drive ("1mpByHLWajydSg5iE223iVWRYfAhndAKA", "VAE", save_dir)

#@markdown 使用云端模ControlNet （By B站 邀月浮白）
shortcut_to_cn = False #@param {type:"boolean"}
if shortcut_to_cn:
  auth.authenticate_user()
  drive_service = build('drive', 'v3')
  save_dir = get_save_dir_id(gcn_dir)
  delete_shortcut_files(save_dir)
  add_shortcut_files_to_drive ("1_sVbogR2El4FM-KvUf4YRNHCB1CpXvgO", "models", save_dir)

In [4]:
#@title 👇 MOD!

import os
import sys
import base64
import binascii

sdw = binascii.unhexlify("737461626c652d646966667573696f6e2d7765627569").decode('ascii')
w = binascii.unhexlify("73642d7765627569").decode('ascii')

webui_dir = f'/content/{sdw}'
gwebui_dir = f'/content/drive/MyDrive/{sdw}'

import pandas as pd
from google.colab import files, drive
!apt -y install -qq aria2 git
mod_name = "mod.xlsx"   #@param {type:"string"}
mod_link = ""   #@param {type:"string"}

use_drive = True  #@param {type:"boolean"}
if use_drive:
    if not os.path.exists('/content/drive'):
        from google.colab import drive
        drive.mount('/content/drive')
        print('Google Drive 已挂载成功！')
else:
    print('Google Drive 未挂载!')

ckpt_dir = f"{webui_dir}/models/Stable-diffusion"
Lora_dir = f"{webui_dir}/models/Lora"
VAE_dir = f"{webui_dir}/models/VAE"
embeddings_dir = f"{webui_dir}/embeddings"
scripts_dir = f"{webui_dir}/scripts"
extensions_dir = f"{webui_dir}/extensions"

if use_drive and os.path.exists(f'/content/drive/MyDrive/{mod_name}'):
    mod_file = f'/content/drive/MyDrive/{mod_name}'
else:
    try:
        uploaded = files.upload()
        mod_file = list(uploaded.keys())[0]
        if not mod_file.endswith('.xlsx'):
            raise ValueError("文件类型错误，必须是.xlsx文件")
    except:
        if mod_link:
            !wget {mod_link} -O /content/{mod_name}
            mod_file = mod_name

df = pd.read_excel(mod_file)

for index, row in df.iterrows():
    file_type = row['类型']
    download_link = row['下载地址']
    file_name = row['名称']

    if file_type == 'CKPT':
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {download_link} -d {ckpt_dir} -o {file_name}
    elif file_type == 'Lora':
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {download_link} -d {Lora_dir} -o {file_name}
    elif file_type == 'VAE':
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {download_link} -d {VAE_dir} -o {file_name}
    elif file_type == 'Extensions':
        repo_name = download_link.split('/')[-1]
        !git clone {download_link} {extensions_dir}/{repo_name}
    elif file_type == 'Embeddings':
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {download_link} -d {embeddings_dir} -o {file_name}
    elif file_type == 'Scripts':
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {download_link} -d {scripts_dir} -o {file_name}

aria2 is already the newest version (1.35.0-1build1).
git is already the newest version (1:2.25.1-1ubuntu3.11).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


Saving wise.xlsx to wise.xlsx

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
a9bbe5|OK  |   102MiB/s|/content/stable-diffusion-webui/models/Stable-diffusion/majicmixRealistic_v5.safetensors

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
4341f3|OK  |    17MiB/s|/content/stable-diffusion-webui/models/Lora/BeautyNwsjMajic2-01.safetensors

Status Legend:
(OK):download completed.
Cloning into '/content/stable-diffusion-webui/extensions/sd-webui-additional-networks.git'...
remote: Enumerating objects: 809, done.
remote: Counting objects: 100% (339/339), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 809 (delta 245), reused 227 (delta 208), pack-reused 470
Receiving objects: 100% (809/809), 269.58 KiB | 663.00 KiB/s, done.
Resolving deltas: 100% (454/454), done.

In [ ]:
#下载模型到 SDwebui （colab）
#下载主模型，请根据自己的需求下载，修改下载地址和文件名即可
!wget https://civitai.com/api/download/models/11745 -O {webui_dir}/models/Stable-diffusion/Chilloutmix-Ni-pruned-fp32-fix.safetensors

#下载 Lora 模型
!wget https://civitai.com/api/download/models/16557 -O {Lora_dir}/ShojoVibev_11.safetensors
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/16557 -d {Lora_dir} -o ShojoVibev_11.safetensors

In [ ]:
#下载模型到 GoogleDrive
#下载链接，文件名，models路径
url = "https://civitai.com/api/download/models/16557" #models文件下载链接
filename = "ShojoVibev_11.safetensors" #保存文件名 .safetensors or .pt
save_dir = f"{gLora_dir}"  # f"{gLora_dir}/Stable-diffusion/"

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -o {filename} -d {save_dir} {url}